In [32]:
import siman
from siman.calc_manage import smart_structure_read
import os
import numpy as np
from tqdm import tqdm
from scipy.constants import physical_constants
POSCARS_DIR_PATH = './siman_rdy'
k_B = physical_constants['Boltzmann constant in eV/K'][0]
FAKE_MAGNETIC_ATOMS = ['Sm', 'Eu']

In [33]:
def count_nn(path_to_poscar, magnetic_atoms):

    """
    calculated the number of nearest neighbors,
    to fit into the Heisenberg model.
    Get a path to POSCAR structure as an input, 
    To avoid errors one should use prettified POSCAR,
    use poscar_prettifier() function first.
    
    Parameters:
        poscar_path(str) - path to the POSCAR file
        magnetic_atoms(list) - two type of atoms to be treated as a magnetic
                            with an opposite spins (up/down).
                            your POSCAR should contain this two types of atoms.
    Returns:
        dict(distance : number_of_neibours) 
    """
    if not os.path.exists(path_to_poscar):
        print(f'File {path_to_poscar} does not exist!')
        return None
    st = smart_structure_read(path_to_poscar)
    st = st.replic([6, 6, 6])
    out = st.nn(i = 1, n = 50, silent=1)
    
    a = list(zip(out['el'][1:], out['dist'][1:]))
    unique_dist = set(round(i[1], 3) for i in a if i[0] in magnetic_atoms) #collect all the unique distances
    magnetic_dist_lst = [(el, round(dist, 3)) for el, dist in a if el in magnetic_atoms]

    dist_neighbNum = {_: 0 for _ in unique_dist} #key- distane, value list of 
                                                #neighbours at 1st 2nd 3d coordination spheres

    for dist in unique_dist:
        for el, distance in magnetic_dist_lst:
            if dist == distance:
                if el == magnetic_atoms[0]:
                    dist_neighbNum[dist] += 1
                elif el == magnetic_atoms[1]:
                    dist_neighbNum[dist] -= 1
    return dist_neighbNum

In [34]:
def get_coefficients(poscars_to_check, up_to=2):
    """
    As a result this function will return coefficients,
    For E0, J1 and J2 for the Hisenber model in a form of matrix.
    Combaining obtained matrix with calculated energies (E_afm) of AFM configurations,
    One can calculate E0, J1, J2, solving a system of linear equations.
    
    Args:
        poscars_to_check - List of paths to POCARs files that should be checked.
        up_to - how many coordinations spheres to check, default check first two.
        
    Example:
        poscars_to_check = ['/EuO/POSCARS2siman/POSCAR_1',
                            '/EuO/POSCARS2siman/POSCAR_2',
                            '/EuO/POSCARS2siman/POSCAR_3']
        get_coefficients(poscars_to_check)
        >>> np.array([[0,  6],
                      [4, -6],
                      [-6,  0]]
    """
    nn_matrix = []
    for path_to_poscar in tqdm(poscars_to_check):
        nn_ls = list(count_nn(path_to_poscar, magnetic_atoms=FAKE_MAGNETIC_ATOMS).values()) #get a list with number of nearest
                                                        #neighbours for the structure
        nn_matrix.append(nn_ls) #add list to he matrix
    nn_matrix = np.array(nn_matrix)
    return nn_matrix[...,:up_to]

In [35]:
def get_exchange_couplings(nn_matrix, energies_afm, spin):
    """
    Returns three float numbers: E0, J1, J2 respectively.
        E0 - is the part of total energy independent of the spin configuration
        J1, J2 - are the first and second nearest neighbor exchange constants
    
    Example:
        nn_matrix = np.array([[ 0  6],
                              [ 4 -6],
                              [-6  0]])

        energies_afm = np.array([-41.614048,
                                 -41.611114,
                                 -41.59929])
                                 
        get_exchange_coeff(nn_matrix, energies_afm)
        >>>-41.609258249999996 -0.00010548412698408951 -5.068518518515268e-05
    """
    nn_matrix = nn_matrix * spin * (spin + 1)
    nn_matrix = np.append(np.ones([len(nn_matrix),1]), nn_matrix, 1)
    E0, J1, J2 = np.linalg.solve(nn_matrix, energies_afm)
    return E0, J1, J2

In [36]:
def calculate_Tc(J1, J2, z1, z2):
    T_c = (J1 * z1 + J2 * z2) / (3 * k_B)
    return T_c

In [37]:
def afm_energies_getter():
    """
    Returns an numpy array with an energies of AntiFerromagnetic configurations,
    calculated by VASP, take as
    """
    return np.array([E1, E2, E3])

In [38]:
def total_num_neighbours(path_to_poscar, magnetic_atoms):
    """
    Return total number of magnetic_atoms in first,
    second and third coordination spheres z1, z2, z3 respectively.
    Since this number is constant for particular structure and will be the
    same for all generated supercells you can use whatever POSCAR file you want.
    
    Args:
        magnetic_atoms(list) - list of atoms which suppose to be magnetic in given structure.
        e.g [Fe, Co] or just [Fe] if there is only one type of magnetic atoms.
    
    Example:
        path_to_poscar = './vasp_inputs/afm3/POSCAR'
        total_num_neighbours(path_to_poscar, magnetic_atoms = ['Eu'])
        >>> (12, 6, 24)
    
    """
    if not os.path.exists(path_to_poscar):
        print(f'File {path_to_poscar} does not exist!')
    st = smart_structure_read(path_to_poscar)
    st = st.replic([6, 6, 6])
    out = st.nn(i = 1, n = 100, silent=1)

    a = list(zip(out['el'][1:], out['dist'][1:]))

    unique_dist = set(round(i[1], 3) for i in a if i[0] in magnetic_atoms) #collect all the unique distances
    magnetic_dist_lst = [round(dist, 3) for el, dist in a if el in magnetic_atoms]

    nn_num_lst = [magnetic_dist_lst.count(dist) for dist in unique_dist]

    z1 = nn_num_lst[0]
    z2 = nn_num_lst[1]
    z3 = nn_num_lst[2]
    return z1, z2, z3

# MAIN

### Initital guess about structure for AFM sp calc

In [10]:
poscars_to_check = [os.path.join(POSCARS_DIR_PATH, f'POSCAR_{i}') for i in range(1, 35)]

In [11]:
for num, nn in enumerate(get_coefficients(poscars_to_check)):
    print(num + 1, nn)

100%|██████████| 34/34 [05:20<00:00,  9.43s/it]

1 [ 0 -6]
2 [-4  6]
3 [-4  6]
4 [ 0 -6]
5 [ 0 -6]
6 [-4  6]
7 [ 0 -6]
8 [6 0]
9 [ 0 -2]
10 [-2  0]
11 [4 2]
12 [ 0 -2]
13 [-2  0]
14 [-4  2]
15 [-2  0]
16 [-2  0]
17 [-4  2]
18 [-2  0]
19 [ 0 -2]
20 [4 2]
21 [-2  0]
22 [ 0 -2]
23 [6 0]
24 [-2  0]
25 [-2  0]
26 [-4  2]
27 [ 0 -2]
28 [6 0]
29 [ 0 -2]
30 [-2  0]
31 [4 2]
32 [-2  0]
33 [-2  0]
34 [-2  0]


# INPUTS

In [39]:
En_dict = {14 : -.83198590E+02 / 2,
1 : -.41611114E+02,
3 : -.41614048E+02,
5 : -.41611123E+02}

id_of_structures = [1, 3, 14]

SPIN=5/2

In [40]:
En_dict

{14: -41.599295, 1: -41.611114, 3: -41.614048, 5: -41.611123}

In [41]:
energies_afm = np.array([En_dict[i] for i in id_of_structures])

poscars_to_check = [os.path.join(POSCARS_DIR_PATH, f'POSCAR_{i}') for i in id_of_structures]

---

In [42]:
z1, z2, z3 = total_num_neighbours(path_to_poscar='./siman_rdy/POSCAR_1', magnetic_atoms=FAKE_MAGNETIC_ATOMS)
print(f'z1 : {z1} , z2 : {z2}, z3 : {z3}')
nn_matrix = get_coefficients(poscars_to_check)
E0, J1, J2 = get_exchange_couplings(nn_matrix, energies_afm, spin=SPIN)
print(f'E0 : {E0} , J1 : {J1}, J2 : {J2}')
Tc = calculate_Tc(J1, J2, z1=z1, z2=z2)

  0%|          | 0/3 [00:00<?, ?it/s]

z1 : 12 , z2 : 6, z3 : 24


100%|██████████| 3/3 [00:16<00:00,  5.65s/it]

E0 : -41.6332435 , J1 : 0.0011807142857143, J2 : 0.00042151428571425445


In [43]:
print(f'{Tc:.1f} K')

64.6 K
